In [1]:
%reload_ext autoreload
%autoreload 2

In [16]:
from jobannotator import annotate
from pathlib import Path
import json
import pigeonXT as pixt

### Load the job data

In [17]:
with Path('../data/job_listings.jsonl').open('r') as f:
    job_data = [json.loads(job_listing) for job_listing in f.readlines()]
print(f'{len(job_data):,} jobs loaded.')

441 jobs loaded.


In [19]:
job_data[:10]

[{'url': 'https://www.jobindex.dk/jobannonce/439053/nordic-content-marketing-lead',
  'text': 'Nordic Content Marketing Lead\n\nToggle navigation\n\nNordic Content Marketing Lead\n\nKonica Minolta is the leading enterprise print solutions provider, and we are on an exciting business transformation journey, where we have created significant growth in IT-Services over the past two years. We are moving from only focusing on print and copy (approx. 80 %) to an increased focus on innovative IT-solutions and services (approx. 20 %). Our clear ambition is to reach a 50/50 split within the next three years. We’re therefore looking for a Nordic Content Marketing lead who has experience with inbound- and content marketing disciplines and can create demand, engage, convert, and retain customers across our digital touch points. \n\nThe position is thought to be placed in Ballerup, Denmark, but if we find the right talent elsewhere, we’re open to hybrid work, or another solution. 70-80% of the acti

### Annotate the job data

In [18]:
annotations = pixt.annotate(job_data,
                            options=['Relevant', 'Irrelevant'],
                            display_fn=lambda x: print(x['text']))

HTML(value='0 of 441 Examples annotated, Current Position: 0 ')

Output()

### Store the job data

In [ ]:
with Path('../data/job_listings.jsonl').open('w') as f:
    for _, row in annotations.iterrows():
        job_listing = row.example
        if row.changed:
            job_listing['label'] = row.label
        f.write(json.dumps(job_listing))